In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
#city_data_df.head()
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [14]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
# Heatmap of percent Cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Heatmap of percent Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Victoria,HK,2021-09-30 23:08:35,22.2855,114.1577,87.22,86,11,1.01
4,4,Cayenne,GF,2021-09-30 23:10:09,4.9333,-52.3333,83.08,78,0,3.00
13,13,Oistins,BB,2021-09-30 23:10:13,13.0667,-59.5333,84.29,70,40,12.66
17,17,Manicore,BR,2021-09-30 23:10:14,-5.8092,-61.3003,80.08,68,90,0.56
22,22,Ciudad Bolivar,VE,2021-09-30 23:10:16,8.1222,-63.5497,84.02,61,98,10.13
27,27,Bin Qirdan,TN,2021-09-30 23:10:17,33.1378,11.2197,75.94,64,0,5.46
29,29,Mabaruma,GY,2021-09-30 23:10:18,8.2000,-59.7833,77.00,85,100,3.89
31,31,High Rock,BS,2021-09-30 23:10:19,26.6208,-78.2833,82.42,72,4,15.35
35,35,Mitsamiouli,KM,2021-09-30 23:10:20,-11.3847,43.2844,75.87,82,47,8.39
38,38,Umm Lajj,SA,2021-09-30 23:10:21,25.0213,37.2685,81.10,66,0,5.88


In [25]:
preferred_cities_df.count()

City_ID       174
City          174
Country       174
Date          174
Lat           174
Lng           174
Max Temp      174
Humidity      174
Cloudiness    174
Wind Speed    174
dtype: int64

In [26]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Victoria,HK,87.22,22.2855,114.1577,
4,Cayenne,GF,83.08,4.9333,-52.3333,
13,Oistins,BB,84.29,13.0667,-59.5333,
17,Manicore,BR,80.08,-5.8092,-61.3003,
22,Ciudad Bolivar,VE,84.02,8.1222,-63.5497,
27,Bin Qirdan,TN,75.94,33.1378,11.2197,
29,Mabaruma,GY,77.00,8.2000,-59.7833,
31,High Rock,BS,82.42,26.6208,-78.2833,
35,Mitsamiouli,KM,75.87,-11.3847,43.2844,
38,Umm Lajj,SA,81.10,25.0213,37.2685,


In [29]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
3,Victoria,HK,87.22,22.2855,114.1577,Mini Hotel Central
4,Cayenne,GF,83.08,4.9333,-52.3333,Hôtel Le Dronmi
13,Oistins,BB,84.29,13.0667,-59.5333,Butterfly Beach Hotel
17,Manicore,BR,80.08,-5.8092,-61.3003,Hotel Durval
22,Ciudad Bolivar,VE,84.02,8.1222,-63.5497,Hotel Edi
27,Bin Qirdan,TN,75.94,33.1378,11.2197,Hôtel Palace
29,Mabaruma,GY,77.00,8.2000,-59.7833,Monty Brooms Guest House
31,High Rock,BS,82.42,26.6208,-78.2833,Hideaway Bahamas Beach Club
35,Mitsamiouli,KM,75.87,-11.3847,43.2844,Foyer ADM
38,Umm Lajj,SA,81.10,25.0213,37.2685,Moon light Furnished Units


In [34]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [36]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))